In [1]:
from stl import mesh
import numpy as np
from IPython.display import clear_output
from multiprocessing.pool import ThreadPool

### Reshaping .stl (s) 

In [2]:
lro_mesh = mesh.Mesh.from_file('../CADs/LRO_35.stl')
for i, _ in enumerate(lro_mesh.vectors):
    lro_mesh.vectors[i] =  lro_mesh.vectors[i]/10
    
lro_mesh.save('../CADs/LRO_35_reduced.stl', mode=stl.Mode.ASCII)  # save as ASCII

NameError: name 'stl' is not defined

### Obstacle modeling

##### 1.  Load ISS point-cloud [ Obtained through CloudCompare]
##### 2. Assign control volume with grid size
##### 3. For all points in the control volume assign  {1 -> Occupied ,0-> Free}
##### 4.  save to .csv for path-planning algorithm development

In [2]:
iss_mesh = mesh.Mesh.from_file('../CADs/Assembled_ISS.stl')
print(iss_mesh.vectors.shape)

(171204, 3, 3)


In [3]:
iss_mesh.vectors[:,0,0].shape

(171204,)

In [4]:
# Get iss sizes and approximate it as a cube
max_x, max_y, max_z = max(iss_mesh.vectors[:,0,0]), max(iss_mesh.vectors[:,0,1]), max(iss_mesh.vectors[:,0,2])
min_x, min_y, min_z = min(iss_mesh.vectors[:,0,0]), min(iss_mesh.vectors[:,0,1]), min(iss_mesh.vectors[:,0,2])

print("max dims:",max_x,max_y, max_z)
print("min dims:",min_x,min_y, min_z)

# center
cx,cy,cz = (max_x + min_x)/2,(max_y + min_y)/2,(max_z + min_z)/2
cube_side = max([(max_x - min_x),(max_y - min_y),(max_z - min_z)])

print(cx,cy,cz)
print(cube_side)

('max dims:', 58.64746, 24.784914, 55.6514)
('min dims:', -25.899393, -18.903086, -55.6514)
(16.374034881591797, 2.9409141540527344, 0.0)
111.3028


In [90]:
# Get point-cloud out of mesh and compute minimum distance between 2 consecutive points
iss_pcl = np.loadtxt("../CADs/iss_pcl.csv")
print(iss_pcl.shape)

d_min = 100
for i,_ in enumerate(iss_pcl[:-1]):
    if np.linalg.norm(iss_pcl[i+1] - iss_pcl[i] ) < d_min:
        d_min = np.linalg.norm(iss_pcl[i+1] - iss_pcl[i] )

print(d_min)
iss_pcl[:3]

(100219, 3)
0.005099019513593094


array([[6.064, 0.434, 5.768],
       [3.222, 1.087, 5.538],
       [3.747, 1.719, 5.299]])

In [91]:
# define a map size 
map_size = 4

# create control cube 
x_span = np.linspace(cx - cube_side/2,cx + cube_side/2, int(cube_side /map_size))
y_span = np.linspace(cy - cube_side/2,cy + cube_side/2, int(cube_side /map_size))
z_span = np.linspace(cz - cube_side/2,cz + cube_side/2, int(cube_side /map_size))

cube_points = []

# create labeled point-cloud

for x in x_span:
    for y in y_span:
        for z in z_span:
            p = np.array([x,y,z]).reshape(3,1)
            cube_points.append(p)
            
           
print(len(cube_points),
      len(iss_pcl))

(19683, 100219)


```
#benchmark
cube_points = cube_points[2 * len(cube_points)/3: 2*len(cube_points)/3+1000]
print(len(cube_points))
```


In [92]:
cp_t1 = cube_points[:int(len(cube_points)/4)]
cp_t2 = cube_points[int(len(cube_points)/4):int(2*len(cube_points)/4)]
cp_t3 = cube_points[int(2*len(cube_points)/4):int(3*len(cube_points)/4)]
cp_t4 = cube_points[int(3* len(cube_points)/4):]

assert(len(cp_t1)+len(cp_t2)+len(cp_t3)+len(cp_t4) == len(cube_points))

def process_cube_points(cube_points, thread_name):
    pctg = 0
    iss_points = []
    free_points = []
    
    for ip,p in enumerate(cube_points):
        occupancy_flag = False
        for p_iss in iss_pcl:
            if np.linalg.norm(p - p_iss) < 1 * np.sqrt(2) * map_size:#1*np.sqrt(2)* map_size
                iss_points.append(p.reshape(1,3).tolist()[0])
                occupancy_flag = True
                #print('bk' + thread_name)
                break

        if not occupancy_flag:
            free_points.append(p.reshape(1,3).tolist()[0])
            
        if( float(ip)/len(cube_points) * 100 > pctg ):
            clear_output()
            print(str(pctg) +"% points evaluated by " + thread_name)
            print("#-iss points:" + str(len(iss_points)))
            pctg += 1
            
    print("DONE FOR THREAD:" + thread_name)
    return [iss_points, free_points]
    



In [93]:
#test on random center-points
process_cube_points([np.array([30,0,0]),np.array([0,0,10]),np.array([0,1,0]),np.array([2,15,0]),np.array([200,0,0])],"&&&")

3% points evaluated by &&&
#-iss points:4
DONE FOR THREAD:&&&


[[[30, 0, 0], [0, 0, 10], [0, 1, 0], [2, 15, 0]], [[200, 0, 0]]]

In [94]:
pool = ThreadPool(processes=16)


r1 =  pool.apply_async(process_cube_points,(cp_t1,'t1',))
r2 =  pool.apply_async(process_cube_points,(cp_t2,'t2',))
r3 =  pool.apply_async(process_cube_points,(cp_t3,'t3',))
r4 =  pool.apply_async(process_cube_points,(cp_t4,'t4',))


rs1 = r1.get()
rs2 = r2.get()
rs3 = r3.get()
rs4 = r4.get()

99% points evaluated by t4
#-iss points:0
DONE FOR THREAD:t1
DONE FOR THREAD:t3
DONE FOR THREAD:t4


In [100]:
print(rs1[0])

[]


In [101]:
iss_points_ = []
free_points_ = []

t_results = [rs1,rs2,rs3,rs4]

for r in t_results:
    ips = r[0]
    fps = r[1]
    for ip in ips:
        iss_points_.append(ip)
    for fp in fps:
        free_points_.append(fp)
    

In [102]:
iss_points = np.array(iss_points_).reshape(len(iss_points_),3)
free_points = np.array(free_points_).reshape(len(free_points_),3)

print iss_points.shape
print free_points.shape

np.savetxt('../MAPs/iss_map_res_'+str(map_size) +'m.csv', iss_points, delimiter=',')
np.savetxt('../MAPs/free_map_res_'+str(map_size) +'m.csv', free_points, delimiter=',')

(6, 3)
(19677, 3)


In [ ]:
iss_map = np.array([0,0,0,1])

for ip in iss_points:
     np.concatenate(np.array([float(ip[0]),float(ip[1]),float(ip[2]),1.])
                    
for ip in free_points:
     np.concatenate(np.array([float(fp[0]),float(fp[1]),float(fp[2]),1.])
                    

    